In [25]:
import pandas as pd
import xlwings as xw

In [26]:
def get_data_with_xlwings(file_name, sheet_name):
    wb = xw.Book(file_name)
    sheet = wb.sheets[sheet_name]
    data = sheet.range('A1').options(pd.DataFrame, expand='table').value
    wb.close()
    return data

In [27]:
def process_data(data):
    #     CREATE ONE FOR GROUND
    service_type_to_sheet_name = {
        "FedEx First Overnight": "FedEx First Overnight®",
        "FedEx Priority Overnight": "FedEx Priority Overnight®",
        "FedEx Standard Overnight": "FedEx Standard Overnight®",
        "FedEx 2Day A.M.": "FedEx 2Day® A.M.",
        "FedEx 2Day": "FedEx 2Day®",
        "FedEx Express Saver": "FedEx Express Saver®",
        "Intra-Hawaii Standard List Rates": "Intra-Hawaii Standard List Rate",
#         BEGIN FREIGHT PRICING MATRIX SECTION
        "FedEx First Overnight Freight": "FedEx First Overnight® Freight",
        "FedEx 1Day Freight": "FedEx 1Day® Freight",
        "FedEx 2Day Freight": "FedEx 2Day® Freight",
        "FedEx 3Day Freight": "FedEx 3Day® Freight",
    }
    mw_service_type_to_sheet_name = {
        "FedEx First Overnight": "MW FedEx First Overnight®",
        "FedEx Priority Overnight": "MW FedEx Priority Overnight®",
        "FedEx Standard Overnight": "MW FedEx Standard Overnight®",
        "FedEx 2Day A.M.": "MW FedEx 2Day® A.M.",
        "FedEx 2Day": "MW FedEx 2Day®",
        "FedEx Express Saver": "MW FedEx Express Saver®",
        "Intra-Hawaii FedEx Priority Overnight": "MWIntra-HawaiiFedExPriorityON",
    }
    
    rates_wb = xw.Book("./separated_us_express_rates.xlsx")
    rate_sheets = rates_wb.sheets
    
    for index, row in data.iterrows():
#         'index' CANNOT BE USED - ITS AN INVOICE NUMBER THAT REPEATS AND INDICES !ACCURATE
#         CONVERTING WEIGHT & ZONE VALUES BELOW TO INTS TO LATER COMBINE AND USE AS COORDINATES
        weight = int(row['RW'])
        zone = int(row['Z'])
        service_type = row['Service Type']
        otpub = row['OTPUB']
        if service_type in service_type_to_sheet_name:
#             BELOW GETS CORRECT PRICING SPREADSHEET AND CONVERTS TO A DATAFRAME
            pricing_matrix_data = rate_sheets[service_type_to_sheet_name[service_type]].range("A1").options(pd.DataFrame, expand='table').value
#             THE NEXT TWO LINES SHOULD BE THE SAME
#             print(f'Found in pricing_matrix_data using RW and zone => {pricing_matrix_data.iloc[weight, zone-2]}')
#             print('OTPUB value that was previously calculated w/o the use of these functions => {otpub}')
        elif service_type == "Ground":
#             DEVELOP A WAY TO HANDLE GROUND SHIPMENTS
            pass
#             BELOW CHANGES CELL TO COLOR RED TO EASILY LOCATE CELLS WE COULDN'T POPULATE
#             otpub.color = (255, 0, 0)  # RGB color for red
        else:
#             BELOW PRINTS ANY SERVICE TYPE FOR WHICH WE MAY HAVE NOT HAVE ANY HANDLING 
            print(service_type)
    
#     IF NOTHING PRINTS, THE ENTIRE CHARGES SHEET WAS PROCESSED AND ALL SERVICE TYPES THAT APPEARED WERE ADDRESSED
        
    rates_wb.close()
    return data

In [28]:
def save_results_with_xlwings(data, file_name, sheet_name):
    print(data)
#     wb = xw.Book(file_name)
#     ws = wb.sheets[sheet_name]
#     ws.range('A1').options(index=False).value = data
#     wb.save()
#     wb.close()

In [29]:
#BEGIN MAIN SCRIPT USING ABOVE FUNCTIONS
file_name = './charges.xlsm'
spreadsheet_name = 'FREIGHT & ACCESSORIALS'
sheet_data = get_data_with_xlwings(file_name, spreadsheet_name)
processed_data = process_data(sheet_data)
# print(processed_data['OTPUB'])
# save_results_with_xlwings(processed_data, file_name, spreadsheet_name)